# Baseline Experiment
- https://drive.google.com/file/d/1sV2DSHaIFE-BvPs0POqH3y4qMH883j6N/view?usp=drive_link
- Hypothesis (h1a): Sentiment outputs, mapped to direction, can predict short-term exchange rate movements - won't perform well because the model lacks domain specific context
- Objective: A baseline comparison before testing hyped-up LLMs capabilities
- Value Proposition: This is the first known study to be conducted on applying language models to trade in emerging currency markets. Especially in a multilingual context.
- Why sentiment analysis as baseline: 
    - FinBERT is widely cited in financial NLP literature
    - Outperforms general BERT and lexicon-based models on tasks like financial sentiment classification
    - Traditional ML methods rely on sparse inputs or static word embeddings (like Word2Vec) which don't capture context
    - Sentiment analysis is commonly used in generating trading signals, however I believe that market does not operate
    on whether a piece of text is happy or sad. Thus, I'm expecting the following experiments to outperform this baseline. 
    I just want to rule sentiment analysis out of the picture. "Predicting directional movement" is a better approach.
    - It was used as a benchmark in very similar paper found at https://doi.org/10.1016/j.mlwa.2023.100508

- Model:
    - HuggingFace Transformers Model: lucas-leme/FinBERT-PT-BR

- Independent Variable (Predictor):
    - Text: headline
    - Category: FinBERT sentiment output
    - Binary Label: heuristic mapping (positive -> 1, negative -> -1) (bullish or bearish in commercial terms)

- Dependent Variable (Ground Truth):
    - Directional Movement: binary direction of exchange rate following news timestamp (exchange rate change after t + 5 minutes)

- Dataset Creation Process:
    - Stage 4 test dataset

- Experimentation:
    - Encoder-only (representation model) - BERT - FinBERT-PT-BR is a domain specific version of FinBERT, another domain specific BERT model
    - Methodology: Load lucas-leme/FinBERT-PT-BR / tokenizer → tokenize headlines → FinBERT → sentiment output

- RESULTS:
    - Bad as expected, check below

Notes:
    - Using HEADLINES ONLY not ARTICLE CONTENT and COMMENTS as past research has shown that these are not useful for prediction purposes, and they are noisy. No timestamps either because that isn't sentiment.

HuggingFace notes
- BERT is an architecture while lucas-leme/FinBERT-PT-BR is a checkpoint
- import the model specific class from the transformers library
- local: call from_pretrained() from the above class to download model's weights (pytorch_model.bin) and config settings (config.json)
- tokenizer is a class from the transformers library that finds the tokenizer specified in the checkpoint and fully preprocesses input text

In [4]:
import pandas as pd
df = pd.read_csv("../../results/exp001/results.csv")
preds = df['Previsão']
print(preds.value_counts(), '\n' 'Total values: 'f'{preds.value_counts().sum()}')

Previsão
-1    219
 1    181
Name: count, dtype: int64 
Total values: 400


In [5]:
mapping = {1: 'Aumento', -1: 'Diminuição'}
df['Previsão'] = df['Previsão'].map(mapping)

In [6]:
from sklearn.metrics import confusion_matrix
'''
[[TP FN]
 [FP TN]]
'''
cm = confusion_matrix(df["Direção"], df["Previsão"], labels=["Aumento", "Diminuição"])
print(cm)

[[ 98 129]
 [ 83  90]]


In [8]:
from sklearn.metrics import classification_report

# Normalize predictions (just in case)
df["Previsão"] = df["Previsão"].str.strip().str.capitalize()
df["Direção"] = df["Direção"].str.strip().str.capitalize()

# Report
report = classification_report(
    df["Direção"], df["Previsão"],
    labels=["Aumento", "Diminuição"],
    target_names=["Aumento", "Diminuição"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

     Aumento      0.541     0.432     0.480       227
  Diminuição      0.411     0.520     0.459       173

    accuracy                          0.470       400
   macro avg      0.476     0.476     0.470       400
weighted avg      0.485     0.470     0.471       400

